# cifar10文件读取

In [1]:
import numpy as np
import pickle
import tensorflow as tf
import time
import math

D:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict


def GetPhoto(pixel):
    assert len(pixel) == 3072
    r = pixel[0:1024]; r = np.reshape(r, [32, 32, 1])
    g = pixel[1024:2048]; g = np.reshape(g, [32, 32, 1])
    b = pixel[2048:3072]; b = np.reshape(b, [32, 32, 1])

    photo = np.concatenate([r, g, b], -1)

    return photo


def GetTrainDataByLabel(label):
    batch_label = []
    labels = []
    data = []
    filenames = []
    for i in range(1, 1+5):
        batch_label.append(unpickle("E:/Data/Cifar-10/cifar-10-python/cifar-10-batches-py/data_batch_%d"%i)[b'batch_label'])
        labels += unpickle("E:/Data/Cifar-10/cifar-10-python/cifar-10-batches-py/data_batch_%d"%i)[b'labels']
        data.append(unpickle("E:/Data/Cifar-10/cifar-10-python/cifar-10-batches-py/data_batch_%d"%i)[b'data'])
        filenames += unpickle("E:/Data/Cifar-10/cifar-10-python/cifar-10-batches-py/data_batch_%d"%i)[b'filenames']

    data = np.concatenate(data, 0)

    label = str.encode(label)
    if label == b'data':
        array = np.ndarray([len(data), 32, 32, 3], dtype=np.int32)
        for i in range(len(data)):
            array[i] = GetPhoto(data[i])
        return array
        pass
    elif label == b'labels':
        return labels
        pass
    elif label == b'batch_label':
        return batch_label
        pass
    elif label == b'filenames':
        return filenames
        pass
    else:
        raise NameError


def GetTestDataByLabel(label):
    batch_label = []
    filenames = []

    batch_label.append(unpickle("cifar-10-python/cifar-10-batches-py/test_batch")[b'batch_label'])
    labels = unpickle("cifar-10-python/cifar-10-batches-py/test_batch")[b'labels']
    data = unpickle("cifar-10-python/cifar-10-batches-py/test_batch")[b'data']
    filenames += unpickle("cifar-10-python/cifar-10-batches-py/test_batch")[b'filenames']

    label = str.encode(label)
    if label == b'data':
        array = np.ndarray([len(data), 32, 32, 3], dtype=np.int32)
        for i in range(len(data)):
            array[i] = GetPhoto(data[i])
        return array
        pass
    elif label == b'labels':
        return labels
        pass
    elif label == b'batch_label':
        return batch_label
        pass
    elif label == b'filenames':
        return filenames
        pass
    else:
        raise NameError

In [3]:
D = GetTrainDataByLabel('data')
L = GetTrainDataByLabel('labels')
data_size = D.shape[0]
print(data_size)
#print(D)
#print(L)

50000


In [4]:
Data = tf.convert_to_tensor(D,tf.float32)
Label = tf.convert_to_tensor(L,tf.float32)

In [5]:
max_steps = 3000
batch_size = 100
image_holder = tf.placeholder(tf.float32, [batch_size, 32, 32, 3])
label_holder = tf.placeholder(tf.int32, [batch_size])

In [6]:
def variable_with_weight_loss(shape, stddev):
    var = tf.Variable(tf.truncated_normal(shape, stddev=stddev))
    return var

In [7]:
weight1 = variable_with_weight_loss(shape=[5, 5, 3, 64], stddev=5e-2)
kernel1 = tf.nn.conv2d(image_holder, weight1, [1, 1, 1, 1], padding='SAME')
bias1 = tf.Variable(tf.constant(0.0, shape=[64]))
conv1 = tf.nn.relu(tf.nn.bias_add(kernel1, bias1))
pool1 = tf.nn.max_pool(conv1, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='SAME')
norm1 = tf.nn.lrn(pool1, 4, bias=1.0, alpha=0.001/9.0, beta=0.75)

weight2 = variable_with_weight_loss(shape=[5, 5, 64, 64], stddev=5e-2)
kernel2 = tf.nn.conv2d(norm1, weight2, [1, 1, 1, 1], padding='SAME')
bias2 = tf.Variable(tf.constant(0.1, shape=[64]))
conv2 = tf.nn.relu(tf.nn.bias_add(kernel2, bias2))
norm2 = tf.nn.lrn(conv2, 4, bias=1.0, alpha=0.001/0.9, beta=0.75)
pool2 = tf.nn.max_pool(norm2, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='SAME')

reshape = tf.reshape(pool2, [batch_size, -1])
dim = reshape.get_shape()[1].value
weight3 = variable_with_weight_loss(shape=[dim, 384], stddev=0.04)
bias3 = tf.Variable(tf.constant(0.1, shape=[384]))
local3 = tf.nn.relu(tf.matmul(reshape, weight3) + bias3)

weight4 = variable_with_weight_loss(shape=[384, 192], stddev=0.04)
bias4 = tf.Variable(tf.constant(0.1, shape=[192]))
local4 = tf.nn.relu(tf.matmul(local3, weight4) + bias4)

weight5 = variable_with_weight_loss(shape=[192, 10], stddev=1/192.0)
bias5 = tf.Variable(tf.constant(0.0, shape=[10]))
logits = tf.add(tf.matmul(local4, weight5), bias5)

In [8]:
def loss(logits, labels):
    labels = tf.cast(labels, tf.int64)
    # 计算logits和labels之间的稀疏softmax交叉熵
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(
        logits=logits, labels=labels, name='cross_entropy_per_example')
    cross_entropy_mean = tf.reduce_mean(cross_entropy, name='cross_entropy')
    # 将cross_entropy的loss添加到整体的loss里
    tf.add_to_collection('losses', cross_entropy_mean)
    return tf.add_n(tf.get_collection('losses'), name='total_loss')

In [9]:
loss = loss(logits, label_holder)
# 优化器选择Adam,学习率选1e-3
train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)
# 求top k的准确率(默认top1即输出分数最高的一类)
top_k_op = tf.nn.in_top_k(logits, label_holder, 1)

In [10]:
# 创建默认的session()
sess = tf.InteractiveSession()
# 初始化全部的模型参数
tf.global_variables_initializer().run()

In [11]:
tf.train.start_queue_runners()

for step in range(max_steps):
    start_time = time.time()
    # 使用sess的run方法执行images_train和labels_train的计算
    start = (step*batch_size)%data_size
    end = min(start+batch_size,data_size)
    image_batch = D[start:end]
    label_batch = L[start:end]
    image_batch = np.reshape(image_batch,(end-start,32,32,3))
    label_batch = np.reshape(label_batch,(end-start))
    #print(image_batch.shape)
    #print(label_batch.shape)
    _, loss_value = sess.run([train_op, loss],feed_dict={image_holder: image_batch, label_holder:label_batch})
    # 记录每个step的时间
    duration = time.time() - start_time
    if step % 10 == 0:
        # 每秒训练的样本数量
        examples_per_sec = batch_size / duration
        # 训练每个batch的时间
        sec_per_batch = float(duration)
        format_str = ('step %d, lass=%.2f (%.1f examples/sec; %.3f sec/batch)')
        print(format_str % (step, loss_value, examples_per_sec, sec_per_batch))

Instructions for updating:
To construct input pipelines, use the `tf.data` module.
step 0, lass=2.31 (53.6 examples/sec; 1.865 sec/batch)
step 10, lass=2.30 (2949.0 examples/sec; 0.034 sec/batch)
step 20, lass=2.31 (2949.0 examples/sec; 0.034 sec/batch)
step 30, lass=2.32 (2949.1 examples/sec; 0.034 sec/batch)
step 40, lass=2.30 (2864.7 examples/sec; 0.035 sec/batch)
step 50, lass=2.12 (2934.5 examples/sec; 0.034 sec/batch)
step 60, lass=2.00 (2949.0 examples/sec; 0.034 sec/batch)
step 70, lass=2.06 (2949.0 examples/sec; 0.034 sec/batch)
step 80, lass=2.12 (2949.0 examples/sec; 0.034 sec/batch)
step 90, lass=2.13 (2932.1 examples/sec; 0.034 sec/batch)
step 100, lass=1.95 (2949.0 examples/sec; 0.034 sec/batch)
step 110, lass=1.93 (2949.1 examples/sec; 0.034 sec/batch)
step 120, lass=1.90 (2949.1 examples/sec; 0.034 sec/batch)
step 130, lass=1.91 (2949.1 examples/sec; 0.034 sec/batch)
step 140, lass=1.78 (2948.3 examples/sec; 0.034 sec/batch)
step 150, lass=1.96 (2864.8 examples/sec; 0.0

step 1320, lass=1.12 (2914.8 examples/sec; 0.034 sec/batch)
step 1330, lass=1.32 (2931.7 examples/sec; 0.034 sec/batch)
step 1340, lass=1.05 (2863.2 examples/sec; 0.035 sec/batch)
step 1350, lass=0.96 (2946.9 examples/sec; 0.034 sec/batch)
step 1360, lass=1.12 (2933.8 examples/sec; 0.034 sec/batch)
step 1370, lass=0.97 (2949.8 examples/sec; 0.034 sec/batch)
step 1380, lass=1.16 (2949.0 examples/sec; 0.034 sec/batch)
step 1390, lass=1.20 (2949.0 examples/sec; 0.034 sec/batch)
step 1400, lass=0.90 (2864.8 examples/sec; 0.035 sec/batch)
step 1410, lass=1.05 (2949.2 examples/sec; 0.034 sec/batch)
step 1420, lass=1.28 (2979.4 examples/sec; 0.034 sec/batch)
step 1430, lass=1.13 (2864.8 examples/sec; 0.035 sec/batch)
step 1440, lass=1.08 (2949.1 examples/sec; 0.034 sec/batch)
step 1450, lass=0.77 (2949.1 examples/sec; 0.034 sec/batch)
step 1460, lass=0.94 (2954.3 examples/sec; 0.034 sec/batch)
step 1470, lass=1.01 (2864.8 examples/sec; 0.035 sec/batch)
step 1480, lass=1.57 (2949.1 examples/se

step 2690, lass=0.87 (2990.7 examples/sec; 0.033 sec/batch)
step 2700, lass=0.91 (2910.6 examples/sec; 0.034 sec/batch)
step 2710, lass=0.65 (2833.6 examples/sec; 0.035 sec/batch)
step 2720, lass=0.91 (2848.8 examples/sec; 0.035 sec/batch)
step 2730, lass=0.71 (2915.0 examples/sec; 0.034 sec/batch)
step 2740, lass=0.79 (2910.3 examples/sec; 0.034 sec/batch)
step 2750, lass=0.91 (2833.9 examples/sec; 0.035 sec/batch)
step 2760, lass=0.70 (2917.4 examples/sec; 0.034 sec/batch)
step 2770, lass=0.99 (2920.8 examples/sec; 0.034 sec/batch)
step 2780, lass=0.90 (2946.1 examples/sec; 0.034 sec/batch)
step 2790, lass=1.00 (2840.6 examples/sec; 0.035 sec/batch)
step 2800, lass=0.96 (3014.6 examples/sec; 0.033 sec/batch)
step 2810, lass=0.88 (2960.5 examples/sec; 0.034 sec/batch)
step 2820, lass=0.99 (2949.1 examples/sec; 0.034 sec/batch)
step 2830, lass=1.11 (2865.2 examples/sec; 0.035 sec/batch)
step 2840, lass=0.86 (2942.3 examples/sec; 0.034 sec/batch)
step 2850, lass=0.75 (2947.7 examples/se